In [1]:
import numpy as np
import tensorflow as tf
%matplotlib qt
import matplotlib.pyplot as plt
np.random.seed(132)

# Non-Linear Regression Setup

In [2]:
def synthetic_data1d(n, alpha):
    x = np.random.uniform(-(3/2)*np.pi, np.pi/2, size=n)
    y = np.cos(x) * np.exp(np.sin(x) + alpha*np.random.normal(size=n))
    return x, y

In [3]:
def synthetic_data2d(n, alpha):
    noise_x = alpha*np.random.normal(size=n)
    noise_y = alpha*np.random.normal(size=n)
    x = np.random.uniform(-2*np.pi, 2*np.pi, size=(n, 2))
    y = np.sin(np.cos(x[:, 1]) + noise_x) + np.exp(np.cos(x[:, 0]) + noise_y)
    return x, y

In [4]:
n1 = 100
n2 = 1000
alpha = 0.05

In [5]:
# 1d example
N = 1000
true_x = np.linspace(-(3 / 2) * np.pi, np.pi / 2, N)
true_y = np.cos(true_x) * np.exp(np.sin(true_x))
x, y = synthetic_data1d(n1, alpha)
plt.figure(1)
plt.scatter(x, y)
plt.plot(true_x, true_y, color='red')
plt.show()

In [6]:
# 2d example
N = 50
xx = np.linspace(-2*np.pi, 2*np.pi, N)
xv, yv = np.meshgrid(xx, xx)
z = np.empty(N*N)
c = 0
for i in range(N):
    for j in range(N):
        z[c] = np.sin(np.cos(yv[i, j])) + np.exp(np.cos(xv[i, j]))
        c += 1

fig = plt.figure(2)
ax = plt.axes(projection='3d')
X, Y = synthetic_data2d(n2, alpha)

ax.scatter(xv.flatten(), yv.flatten(), z, color='black', alpha=0.2)
# ax.scatter(X[:,0], X[:,1], Y, color='red')
plt.show()

In [7]:
# Rescale data between 0 and 1
data_max = np.max(X)
data_min = np.min(X)
X = (X - data_min) / (data_max - data_min)
# Split up data
train_split = 0.8
batch_size = 64
num_data = len(Y)
train_x = X[0:int(num_data*train_split), :]
train_y = Y[0:int(num_data*train_split)]
test_x = X[int(num_data*train_split):, :]
test_y = Y[int(num_data*train_split):]
# convert to tensors
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))
# shuffle and batch
train_dataset = train_dataset.shuffle(int(num_data*train_split)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

2022-08-04 08:21:10.564944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Serial Neural Network Results

In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(2,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [9]:
def scheduler(epoch, lr):
  if epoch < 45:
    return lr
  else:
    return lr * tf.math.exp(-0.05)

In [10]:
learning_rate = 0.01
opt = tf.keras.optimizers.Adam(learning_rate)
lossF = tf.keras.losses.MeanSquaredError()
model.compile(loss=lossF, optimizer=opt)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model.fit(train_dataset, epochs=150, callbacks=[callback])

Epoch 1/150
13/13 [==============================] - 0s 2ms/step - loss: 1.3434 - lr: 0.0100
Epoch 2/150
13/13 [==============================] - 0s 2ms/step - loss: 1.0149 - lr: 0.0100
Epoch 3/150
13/13 [==============================] - 0s 2ms/step - loss: 0.9857 - lr: 0.0100
Epoch 4/150
13/13 [==============================] - 0s 1ms/step - loss: 0.9893 - lr: 0.0100
Epoch 5/150
13/13 [==============================] - 0s 2ms/step - loss: 0.9420 - lr: 0.0100
Epoch 6/150
13/13 [==============================] - 0s 2ms/step - loss: 0.8827 - lr: 0.0100
Epoch 7/150
13/13 [==============================] - 0s 2ms/step - loss: 0.8435 - lr: 0.0100
Epoch 8/150
13/13 [==============================] - 0s 2ms/step - loss: 0.8091 - lr: 0.0100
Epoch 9/150
13/13 [==============================] - 0s 2ms/step - loss: 0.8137 - lr: 0.0100
Epoch 10/150
13/13 [==============================] - 0s 2ms/step - loss: 0.7749 - lr: 0.0100
Epoch 11/150
13/13 [==============================] - 0s 2ms/step - l

In [11]:
model.evaluate(test_dataset)

4/4 [==============================] - 0s 1ms/step - loss: 0.0276


0.027639135718345642

In [12]:
# test_predictions = model.predict(test_dataset).flatten()
X2 = np.vstack((xv.flatten(), yv.flatten())).transpose()
X2 = (X2 - np.min(X2)) / (np.max(X2) - np.min(X2))
test_predictions = model.predict(X2).flatten()
plt.figure(3)
plt.scatter(z, test_predictions, color='blue')
a = min(np.min(z), np.min(test_predictions)) - 0.5
b = max(np.max(z), np.max(test_predictions)) + 0.5
e = np.linspace(a,b,100)
plt.xlim(a, b)
plt.ylim(a,b)
plt.plot(e,e, color='black')
plt.show()

79/79 [==============================] - 0s 761us/step


In [13]:
fig = plt.figure(4)
ax = plt.axes(projection='3d')
ax.scatter(X2[:,0], X2[:,1], z, color='black', alpha=0.1)
ax.scatter(X2[:,0], X2[:,1], test_predictions, color='red')
plt.show()